# Topical Expert Model

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import operator
import os
print(os.getcwd())
print(os.listdir('/Users/Brandon/Yelp Dataset/dataset'))
import json
import glob
from datetime import datetime
from pandas import HDFStore,DataFrame
import operator

/Users/Brandon/Yelp Dataset/Yelp-FA17/local-elites/brandon
['.DS_Store', 'business.json', 'checkin.json', 'photos.json', 'review.json', 'review5000.json', 'tip.json', 'user.json', 'user5000.json']


In [2]:
def load_json_to_df(datapass):
    '''
    Load the json file and parse the file to pandas dataframe format
    
    Input:
        datapass(str) : directory to the json file
    Output:
        df(dataframe) : pandas dataframe object
    '''
    
    data = [] 
    with open(datapass) as data_file: 
        for f in data_file:
            data.append(json.loads(f))
    df = pd.DataFrame(data)
    return df

LOAD THE DATA

In [3]:
%%time
import_file = "/Users/Brandon/Yelp Dataset/dataset/review.json"
review = load_json_to_df(import_file)

CPU times: user 1min 3s, sys: 23.3 s, total: 1min 26s
Wall time: 1min 38s


In [4]:
%%time
import_file = "/Users/Brandon/Yelp Dataset/dataset/business.json"
business = load_json_to_df(import_file)

CPU times: user 5.25 s, sys: 453 ms, total: 5.7 s
Wall time: 6.06 s


In [5]:
%%time
import_file = "/Users/Brandon/Yelp Dataset/dataset/user.json"
user = load_json_to_df(import_file)

CPU times: user 53.3 s, sys: 17.5 s, total: 1min 10s
Wall time: 1min 21s


Sort by category

In [24]:
def categoryFind(df, cat):
    return [cat in i for i in df['categories']]

def category_counts(df):
    all_cats = []
    for i in df['categories']:
        for j in i:
            all_cats.append(j)
    categories = set(all_cats)
    category_counts = {}
    for cat in categories:
        category_counts[cat] = all_cats.count(cat)
    return len(categories), category_counts

def top_categories(counts, n):
    top_categories = sorted(counts.keys(), key=(lambda k: counts[k]),reverse=True)[:n]
    return top_categories

In [8]:
num_cats, cat_counts = category_counts(business)
top_cats = top_categories(cat_counts, 100)
top_cats

['Restaurants',
 'Shopping',
 'Food',
 'Beauty & Spas',
 'Home Services',
 'Health & Medical',
 'Nightlife',
 'Bars',
 'Automotive',
 'Local Services',
 'Event Planning & Services',
 'Active Life',
 'Fashion',
 'Sandwiches',
 'Fast Food',
 'American (Traditional)',
 'Pizza',
 'Coffee & Tea',
 'Hair Salons',
 'Hotels & Travel',
 'Arts & Entertainment',
 'Home & Garden',
 'Auto Repair',
 'Italian',
 'Burgers',
 'Doctors',
 'Breakfast & Brunch',
 'Mexican',
 'Nail Salons',
 'Professional Services',
 'American (New)',
 'Chinese',
 'Real Estate',
 'Specialty Food',
 'Fitness & Instruction',
 'Pets',
 'Grocery',
 'Bakeries',
 'Cafes',
 'Hair Removal',
 'Dentists',
 'Hotels',
 'Desserts',
 'Skin Care',
 "Women's Clothing",
 'Education',
 'Japanese',
 'Ice Cream & Frozen Yogurt',
 'Pet Services',
 'Day Spas',
 'Massage',
 'General Dentistry',
 'Financial Services',
 'Pubs',
 'Chicken Wings',
 'Seafood',
 'Contractors',
 'Salad',
 'Gyms',
 'Sushi Bars',
 'Sports Bars',
 'Apartments',
 'Caterers

In [9]:
%%time
businessReviews = pd.merge(review, restaurants,on='business_id',right_index=True,sort=False)[['business_id', 'cool', 'funny', 'review_id', 'stars_x', 'text', 'useful', 'user_id', 'categories', 'review_count', 'stars_y']]
bus_rev_users = pd.merge(businessReviews, user,on='user_id',right_index=True,sort=False)[['business_id', 'cool_x', 'funny_x', 'review_id', 'stars_x', 'text', 'useful_x', 'user_id', 'categories', 'stars_y', 'average_stars', 'cool_y', 'elite', 'fans', 'friends', 'name', 'review_count_y', 'yelping_since']]

CPU times: user 38 s, sys: 28.2 s, total: 1min 6s
Wall time: 1min 37s


In [10]:
print(len(businessReviews))
print(len(bus_rev_users))
print(list(businessReviews))
print(list(bus_rev_users))

2927731
2927731
['business_id', 'cool', 'funny', 'review_id', 'stars_x', 'text', 'useful', 'user_id', 'categories', 'review_count', 'stars_y']
['business_id', 'cool_x', 'funny_x', 'review_id', 'stars_x', 'text', 'useful_x', 'user_id', 'categories', 'stars_y', 'average_stars', 'cool_y', 'elite', 'fans', 'friends', 'name', 'review_count_y', 'yelping_since']


## Feature Extraction From Yelp Data

In [11]:
def numCategoryReviews(businessReviews):
    return businessReviews[['business_id','user_id']].groupby('user_id',as_index=False).count()

def average_star_category_rating(businessReviews):
    return businessReviews[['user_id','stars_x']].groupby('user_id',as_index=False).mean()

def std_star_category_rating(businessReviews):
    busRev = businessReviews[['user_id','stars_x']].groupby('user_id',as_index=False).agg(np.var)
    busRev['stars_x'] = busRev['stars_x'].pow(1./2)
    return busRev

def funny_useful_cool(businessReviews):
    return businessReviews[['user_id','funny','useful','cool']].groupby('user_id',as_index=False).sum()

def months_yelping(businessReviews):
    user_dates = businessReviews[['user_id','yelping_since']].drop_duplicates()
    curr_date = datetime(2017,10,17)
    uyelp_dates = [datetime.strptime(i, "%Y-%m-%d") for i in user_dates['yelping_since']]
    months_yelping = [(curr_date.year - i.year) * 12 + curr_date.month - i.month for i in uyelp_dates]
    user_dates['yelping_since'] = months_yelping
    return user_dates

def get_elite_users(df):
    temp = []
    for i in df['elite'].index.values:
        if df['elite'][i]:
            temp.append(i)
    return temp, df.loc[temp]

## Define the first feature model to be analyzed
Features: Total Reviews by User, Number of Category Reviews, Average Rating in Category, Std Dev of Ratings in Category, Funny, Useful, and Cool Votes, Number of Months since joining Yelp.

In [25]:
def feature1(df, df1):
    tot_reviews = df1[['user_id','review_count_y']]
    tot_reviews.columns = ['user_id','Total Reviews by User']
    catRev = numCategoryReviews(df)
    catRev.columns = ['user_id','Num Category Reviews']
    averageCat = average_star_category_rating(df)
    averageCat.columns = ['user_id','Average Rating in Category']
    stdCat = std_star_category_rating(df)
    stdCat.columns = ['user_id','Std Dev of Ratings in Category']
    fuc = funny_useful_cool(df)
    monthsYelp = months_yelping(df1)
    monthsYelp.columns = ['user_id', 'Months Yelping']
    is_elite = df1[['user_id','elite']]
    features = tot_reviews.merge(catRev,on='user_id').merge(averageCat,on='user_id').merge(stdCat,on='user_id').merge(fuc,on='user_id').merge(monthsYelp,on='user_id').merge(is_elite,on='user_id')
    features = features.drop_duplicates('user_id')
    features.index = range(len(features))
    elite_ind, elite_users = get_elite_users(features)
    expertClassifier = [0]*len(features)
    for i in elite_ind:
        expertClassifier[i]=1
    features = pd.concat([features, pd.DataFrame(expertClassifier,columns=['is_expert'])],axis=1)
    return features.fillna(0)

In [26]:
%%time
feature_set_1 = feature1(businessReviews, bus_rev_users)

CPU times: user 1min 30s, sys: 59.8 s, total: 2min 29s
Wall time: 2min 58s


In [27]:
print(len(feature_set_1))
feature_set_1.head(5)

823317


,user_id,Total Reviews by User,Num Category Reviews,Average Rating in Category,Std Dev of Ratings in Category,funny,useful,cool,Months Yelping,elite,is_expert
0,kzyLOqiJvyw_FWFTw2rjiQ,2,2,3.000000,2.828427,0,0,0,16,[],0
1,WZXp9-V2dqRRJqhGgRqueA,327,44,3.477273,0.762146,42,54,46,61,"[2016, 2015]",1
2,XylT12exfdLiI_3uDLVIpw,344,31,3.741935,1.504831,11,32,13,84,[],0
3,Ji9PeffxjwqPLO7pEfSpKQ,93,58,3.258621,1.101201,49,98,68,107,"[2010, 2009]",1
4,TLIWzAJPrET0zX4_vgvLhg,53,19,3.631579,0.830698,1,4,0,38,[],0


## Classifier Setups (Sklearn Library)*

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import tree

In [29]:
'''Runs the model specified by the clf (classifier), and features.
Features is assumed to have an 'is_expert' column which gives the classifications'''
def run_model(clf, features):
    train, test = train_test_split(X)
    train_classifier = train['is_expert'].values
    test_classifier = test['is_expert'].values
    train = train.drop('is_expert',axis=1)
    test = test.drop('is_expert', axis=1)
    clf.fit(train, train_classifier)
    rf_pred = clf.predict(test)
    model_perf= {'Model_Score' : clf.score(test, test_classifier),
                'Predictions' : rf_pred,
                'Prediction Probabilities' : clf.predict_proba(test),
                'Total_Tested' : len(rf_pred),
                'Num_Experts_Predicted' : sum(rf_pred),
                'Num_Experts_Actual' : sum(test_classifier),
                'Num_Experts_Training' : sum(train_classifier)} 
    return model_perf

'''Runs the model n times, and prints out a dictionary with the statistics'''
def bootstrap_model(clf, features, n):
    models = []
    for i in range(1,n):
        models.append(run_model(clf, features))
    return models

'''Gets statistics from the bootstrap list of dictionaries'''
def boot_statistics(models):
    stats = []
    expert_pred_percentage = [i['Num_Experts_Predicted']/i['Num_Experts_Actual'] for i in RF_bootstrap]
    stats = expert_pred_percentage
    return stats

def graph_from_statistics(stats):
    

 #### Drop unnessary columns from the features table (call it X).

In [30]:
X = feature_set_1.drop('user_id',axis=1).drop('elite',axis=1)

## Run a random forest classifier

In [36]:
clf_RF = RandomForestClassifier(max_depth=3)
run_model(clf_RF,X)

{'Model_Score': 0.95621143662245545,
 'Num_Experts_Actual': 12682,
 'Num_Experts_Predicted': 6163,
 'Num_Experts_Training': 37997,
 'Prediction Probabilities': array([[ 0.98200494,  0.01799506],
        [ 0.98457804,  0.01542196],
        [ 0.98457804,  0.01542196],
        ..., 
        [ 0.9379149 ,  0.0620851 ],
        [ 0.98457804,  0.01542196],
        [ 0.98457804,  0.01542196]]),
 'Predictions': array([0, 0, 0, ..., 0, 0, 0]),
 'Total_Tested': 205830}

In [32]:
RF_bootstrap = bootstrap_model(clf_RF, X, 10)

In [37]:
RF_bootstrap

[{'Model_Score': 0.95601224311324884,
  'Num_Experts_Actual': 12437,
  'Num_Experts_Predicted': 5773,
  'Num_Experts_Training': 38242,
  'Prediction Probabilities': array([[ 0.46175546,  0.53824454],
         [ 0.98377684,  0.01622316],
         [ 0.9803217 ,  0.0196783 ],
         ..., 
         [ 0.97055319,  0.02944681],
         [ 0.98716111,  0.01283889],
         [ 0.98716111,  0.01283889]]),
  'Predictions': array([1, 0, 0, ..., 0, 0, 0]),
  'Total_Tested': 205830},
 {'Model_Score': 0.9548850993538357,
  'Num_Experts_Actual': 12637,
  'Num_Experts_Predicted': 5815,
  'Num_Experts_Training': 38042,
  'Prediction Probabilities': array([[ 0.879875  ,  0.120125  ],
         [ 0.85845812,  0.14154188],
         [ 0.98607129,  0.01392871],
         ..., 
         [ 0.98607129,  0.01392871],
         [ 0.98607129,  0.01392871],
         [ 0.94819677,  0.05180323]]),
  'Predictions': array([0, 0, 0, ..., 0, 0, 0]),
  'Total_Tested': 205830},
 {'Model_Score': 0.94835543895447705,
  'Num_

## Run a gaussian naive bayes classifier

In [35]:
from sklearn.naive_bayes import GaussianNB
clf_NB = GaussianNB()
run_model(clf_NB,X)

{'Model_Score': 0.95621143662245545,
 'Num_Experts_Actual': 12732,
 'Num_Experts_Predicted': 8569,
 'Num_Experts_Training': 37947,
 'Prediction Probabilities': array([[  9.99999963e-01,   3.70040583e-08],
        [  9.99999996e-01,   3.98915138e-09],
        [  9.99999809e-01,   1.90917408e-07],
        ..., 
        [  9.99999980e-01,   1.99553668e-08],
        [  9.99999731e-01,   2.68650930e-07],
        [  9.99999976e-01,   2.41525317e-08]]),
 'Predictions': array([0, 0, 0, ..., 0, 0, 0]),
 'Total_Tested': 205830}

## Run a decision tree classifier

In [34]:
clf_DT = tree.DecisionTreeClassifier()
run_model(clf_DT, X)

{'Model_Score': 0.95333527668464269,
 'Num_Experts_Actual': 12581,
 'Num_Experts_Predicted': 12670,
 'Num_Experts_Training': 38098,
 'Prediction Probabilities': array([[ 1.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        ..., 
        [ 1.,  0.],
        [ 1.,  0.],
        [ 1.,  0.]]),
 'Predictions': array([0, 0, 0, ..., 0, 0, 0]),
 'Total_Tested': 205830}